# Scipy: A fundamental library for scientific computing

https://docs.scipy.org/doc/scipy/reference/tutorial/index.html

In [ ]:
import numpy as np
import scipy as sp

np.info(sp)

In [ ]:
import scipy.optimize
np.info(sp.optimize)

In [ ]:
import scipy.integrate
np.info(sp.integrate)

In [ ]:
import scipy.special
np.info(sp.special)

In [ ]:
import scipy.linalg
np.info(sp.linalg)

# Integración.

El paquete scipy.integrate provee muchas tecnicas de integracion, incluyendo integración de EDOs. 

 Methods for Integrating Functions given function object.

       quad          -- General purpose integration.

       dblquad       -- General purpose double integration.

       tplquad       -- General purpose triple integration.

       fixed_quad    -- Integrate func(x) using Gaussian quadrature of order n.

       quadrature    -- Integrate with given tolerance using Gaussian quadrature.

       romberg       -- Integrate func using Romberg integration.

 Methods for Integrating Functions given fixed samples.

       trapz         -- Use trapezoidal rule to compute integral from samples.

       cumtrapz      -- Use trapezoidal rule to cumulatively compute integral.

       simps         -- Use Simpson's rule to compute integral from samples.

       romb          -- Use Romberg Integration to compute integral from
                        (2**k + 1) evenly-spaced samples.

       See the special module's orthogonal polynomials (special) for Gaussian
          quadrature roots and weights for other weighting factors and regions.

 Interface to numerical integrators of ODE systems.

       odeint        -- General integration of ordinary differential equations.

       ode           -- Integrate ODE using VODE and ZVODE routines.

La función 'quad' integra funciones de una variable entre dos puntos. Por ejemplo: 

In [ ]:
# Importamos
import scipy as sp
import scipy.integrate as integrate
import scipy.special as special

In [ ]:
special.beta?

In [ ]:
special.beta(1, 3)

In [ ]:
sp.integrate.quad(lambda x: sp.special.beta(1, x), 2, 4)

In [ ]:
# Ejemplo, integrar la función de bessel.
result, error = integrate.quad(lambda x: special.jv(2.5, x), 0, 4.5)
result, error

In [ ]:
from numpy import sqrt, sin, cos, pi
I = sqrt(2 / pi) * (18.0 / 27 * sqrt(2) * cos(4.5) - 4.0 / 27 * sqrt(2)
                    * sin(4.5) + sqrt(2 * pi) * special.fresnel(3 / sqrt(pi))[0])
print(abs(result - I))

'quad' recibe tres argumentos, el primero de ellos es una función, y los dos siguientes son los límites de integración.
Retorna una tupla, cuyo primer elemento estima el valor de la integral, y el segundo es una cota superior para el error.

Cuando la función recibe parámetros adicionales, se pueden proveer en el argumento 'args' 

por ejemplo, para calcular

$I(a,b) = \int_0^1 (ax^2 + b) dx$

puede ser evaluada de la siguiente forma:


In [ ]:
from scipy.integrate import quad


def integrand(x, a, b):
    return a * x**2 + b


a = 2
b = 1
I = quad(integrand, 0, 1, args=(a, b))
I

### Integración multiple

Los mecanismos de integración doble y triple están en las funciones 'dblquad' y 'tplquad'. Estas funciones reciben la función a integrar y cuatro, o seis, argumentos respectivamente. Los límites de todas las integrales internas necesitan ser definidas como funciones.

Un ejemplo se presenta a continuación para integrar

$I = \int_{y=0}^{1/2} \int_{x=0}^{1-2y} (xy) dxdy$


In [ ]:
from scipy.integrate import dblquad

area = dblquad(lambda x, y: x * y, 0, 0.5, lambda x: 0, lambda y: 1 - 2 * y)
area

## Optimización

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html
El paquete scipy.optimize provee algoritmos comunmente usados en optimización, entre ellas:
- Minimización con o sin restricciones para funciones escalares (métodos BFGS, Newton conjugate gradient, Nelder-Mead Simplex, etc)
- Rutinas de fuerza bruta
- Mínimos cuadrados y algoritmos de ajuste de curvas


En el ejemplo que se muestra a continuación se minimiza la función de Rosenbrok de dos variables.
El algoritmo usado puede ser 'nelder-mead' o bien 'powell'. Lo bueno de estos métodos es que sólo se necesita evaluar la función y no usa evaluación del gradiente. (es por ello que es más lenta).

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
def rosen(x):
    return sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

In [ ]:
res = minimize(rosen, x0, options={'maxiter': 100, 'disp': True})
print(res)

In [ ]:
res = minimize(rosen, x0, method='powell', options={'xtol': 1e-8, 'disp': True})
print(res)

In [ ]:
rosen(res.x)

In [ ]:
res = minimize(rosen, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
print(res)

Para obtener un mejor desempeño (converger en menos iteraciones al óptimo), se implementará un método para optimizar que usa el gradiente de la función, por ello se define la función rosen_der, que corresponde al gradiente.

Luego se llama al algoritmo 'BFGS', el cual recibe como argumento el gradiente de la función en 'jac'.

En caso de no recibir explicitamente el jacobiano, el método lo obtiene numéricamente por diferencias finitas.

Para minimizaciones de problemas más grandes, se puede usar el método de Newton-CG, el cual necesita el Hessiano. No se explicita aquí.

In [ ]:
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

In [ ]:
res = minimize(rosen, x0, method='CG', jac=rosen_der, options={'disp': True})
print(res)

In [ ]:
res = minimize(rosen, x0, method='BFGS', jac=rosen_der, options={'disp': True})
print(res)

In [ ]:
res = minimize(rosen, x0, method='L-BFGS-B', jac=rosen_der, options={'disp': True})
print(res)

In [ ]:
res = minimize(rosen, x0, method='Newton-CG', jac=rosen_der, options={'disp': True})
print(res)

## Minimización con restricciones de funciones escalares multivariables 

La función minimize también posee algoritmos de optimización con restricciones. El algoritmo Sequential Least Squares Programming se considerará acá, el cual permite trabajar con minimización con restricciones de la forma:


\begin{align}
\text{min} \quad       &    f(x)\\
\text{subject to }  & C_i(x) = 0, \  i=1,...MEQ,\\
           & C_j(x) \geq 0,\ j=MEQ+1,...,M\\           
           & x_L \leq x \leq x_U, 
\end{align}
           
Por ejemplo, consideremos el problema de maximizar la funcion $f(x, y) = 2 x y + 2 x - x^2 - 2 y^2$

sujeta a  $x^3 - y = 0,\ y-1 \geq 0$.

Se presenta la función y su derivada:

In [ ]:
def func(x, sign=1.0):
    """ Objective function """
    return sign * (2 * x[0] * x[1] + 2 * x[0] - x[0]**2 - 2 * x[1]**2)


def func_deriv(x, sign=1.0):
    """ Derivative of objective function """
    dfdx0 = sign * (-2 * x[0] + 2 * x[1] + 2)
    dfdx1 = sign * (2 * x[0] - 4 * x[1])
    return np.array([dfdx0, dfdx1])

El parámetro signo se introduce para multiplicar por -1 para minimizar o maximizar la función.

Las restricciones se definen como una secuencia de diccionarios, con keys: 'type', 'fun', y 'jac'.

In [ ]:
cons = ({'type': 'eq',
          'fun' : lambda x: np.array([x[0]**3 - x[1]]),
          'jac' : lambda x: np.array([3.0*(x[0]**2.0), -1.0])},
         {'type': 'ineq',
          'fun' : lambda x: np.array([x[1] - 1]),
          'jac' : lambda x: np.array([0.0, 1.0])})

In [ ]:
# minimización sin restriccion
res = minimize(func, [-1.0,1.0], args=(-1.0,), jac=func_deriv, method='SLSQP', options={'disp': True})

print(res.x)

In [ ]:
# minimizacion con restriccion
res = minimize(func, [-1.0,1.0], args=(-1.0,), jac=func_deriv, constraints=cons, method='SLSQP', options={'disp': True})

print(res.x)

## Minimización de minimos cuadrados

Scipy es capaz de resolver problemas no lineales de minimos cuadrados sujetos a restricciones de la forma

\begin{align}
\text{min } \frac{1}{2} \sum_{1=1}^{m} \rho (f_i(x)^2)\\
\text{subject to } l_b \leq x \leq u_b
\end{align}
donde $f_i$ son funciones suaves de $R^N$ a $R$. El proposito de $\rho$ es reducir la infuencia de outliers.

Todos los métodos específicos de minimos cuadrados utilzian la matriz Jacobiana de $f$, y es muy recomendado computar esta matriz y pasarsela 'least_squares', de otra forma será estimado numericamente.

Se considerará el 'Analysis of an Enzyme Reaction'

In [ ]:
from scipy.optimize import least_squares


def model(x, u):
    return x[0] * (u ** 2 + x[1] * u) / (u ** 2 + x[2] * u + x[3])


def fun(x, u, y):
    return model(x, u) - y


def jac(x, u, y):
    J = np.empty((u.size, x.size))
    den = u ** 2 + x[2] * u + x[3]
    num = u ** 2 + x[1] * u
    J[:, 0] = num / den
    J[:, 1] = x[0] * u / den
    J[:, 2] = -x[0] * num * u / den ** 2
    J[:, 3] = -x[0] * num / den ** 2
    return J


u = np.array([4.0, 2.0, 1.0, 5.0e-1, 2.5e-1, 1.67e-1,
              1.25e-1, 1.0e-1, 8.33e-2, 7.14e-2, 6.25e-2])
y = np.array([1.957e-1, 1.947e-1, 1.735e-1, 1.6e-1, 8.44e-2,
              6.27e-2, 4.56e-2, 3.42e-2, 3.23e-2, 2.35e-2, 2.46e-2])
x0 = np.array([2.5, 3.9, 4.15, 3.9])
res = least_squares(fun, x0, jac=jac, bounds=(0, 100), args=(u, y), verbose=1)

In [ ]:
res.x

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

u_test = np.linspace(0, 5)
y_test = model(res.x, u_test)

plt.plot(u, y, 'o', markersize=4, label='data')
plt.plot(u_test, y_test, label='fitted model')
plt.xlabel("u")
plt.ylabel("y")
plt.legend(loc='lower right')
plt.show()

## Interpolación

Hay muchas facilidades para interpolación en Scipy, para datos en 1,2 o más dimensiones.
- Una clase que interpola en 1D es 'interp1d', el cual ofrece muchos métodos de interpolación.
- La función 'griddata' ofrece una simple interfaz de interpolación para N dimensiones.
- Splines cúbicas para interpolar en 1 ó 2 dimensiones.

In [ ]:
# Interpolación en 1D
from scipy.interpolate import interp1d

La función 'interp1d' es un método conveniente para crear una función basada en datos fijos que puede ser evaluada en cualquier punto al interior del dominio dados los datos usando interpolación lineal.

El siguiente ejemplo demuestra su uso, para el caso de interpolación lineal y spline cúbico:

In [ ]:
x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2 / 9.0)
plt.scatter(x, y)
plt.show()

In [ ]:
f = interp1d(x, y)  # linear
xx = np.linspace(0, 10, num=1000, endpoint=True)

plt.plot(x, y, '*r', xx, f(xx), 'g')
plt.show()

In [ ]:

f2 = interp1d(x, y, kind='cubic')  # quadratic cubic nearest
xx = np.linspace(0, 10, num=1000, endpoint=True)

plt.plot(x, y, '*r', xx, f2(xx), 'b')
plt.show()

## Interpolación multivariada

Para el caso de tener datos multidimensionales, por ejemplo para una función subyacente f(x,y) en la que sólo se conoce su valor en puntos (x_i, y_i) que no forman una grilla regular.

Supongamos que queremos interpolar la función 2D en una grilla [0,1]x[0,1]

In [ ]:
def func(x, y):
    return x * (1 - x) * np.cos(4 * np.pi * x) * np.sin(4 * np.pi * y**2)**2


grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]

En la que se conocen su valor en 1000 puntos

In [ ]:
points = np.random.rand(1000, 2)
values = func(points[:,0], points[:,1])

Para interpolar se usa la función griddata, y se usan diferentes métodos

In [ ]:
from scipy.interpolate import griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

Y se grafican 

In [ ]:
plt.subplot(221)
plt.imshow(func(grid_x, grid_y).T, extent=(0, 1, 0, 1), origin='lower')
#plt.plot(points[:, 0], points[:, 1], 'k.', ms=1)
plt.title('Original')

plt.subplot(222)
plt.imshow(grid_z0.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Nearest')

plt.subplot(223)
plt.imshow(grid_z1.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Linear')

plt.subplot(224)
plt.imshow(grid_z2.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Cubic')

plt.gcf().set_size_inches(6, 6)
plt.show()

## Interpolación Spline

La interpolación spline requiere dos pasos escenciales: 1. una representación spline de la curva es computada y 2. la spline se evalúa en los puntos deseados. Para encontrar la representación spline, hay dos formas distintas de hacerlo: directa y paramétrica.
El método directo encuentra la representación spline de una curva en un espacio plano dos-dimensional usando la función 'splrep'. 

Para curvas en espacios N-dimensionales, la función 'spl
prep' permite definir la curva paramétricamente.

El parámetro 's' es usado para especificar el grado de suavidad a considerar durante el ajuste de la curva. Si no se desea suavidad, el valor 's=0' debe ser especificado.

Para evaluar la spline se usa 'splev', y sus derivadas se evalúan con 'splev' y 'spalde'.

A continuación se muestra un uso.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [ ]:
x = np.arange(0, 2 * np.pi + np.pi / 4, 2 * np.pi / 8)
y = np.sin(x)

tck = interpolate.splrep(x, y, s=0)
xnew = np.arange(0, 2 * np.pi, np.pi / 50)
ynew = interpolate.splev(xnew, tck, der=0)
# help(interpolate.splev0)

plt.figure()
plt.plot(x, y, 'x')
plt.plot(xnew, np.sin(xnew), 'b--', linewidth=2)
plt.plot(xnew, ynew, 'y')
plt.legend(['Linear', 'Cubic Spline', 'True'])
plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Cubic-spline interpolation')
plt.show()

Y la derivada de la curva:

In [ ]:
yder = interpolate.splev(xnew, tck, der=1)
plt.figure()
plt.plot(xnew, yder, xnew, np.cos(xnew), '--')
plt.legend(['Cubic Spline', 'True'])
plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Derivative estimation from spline')
plt.show()

La integral de la spline

In [ ]:
def integ(x, tck, constant=-1):
    x = np.atleast_1d(x)
    out = np.zeros(x.shape, dtype=x.dtype)
    for n in range(len(out)):
        out[n] = interpolate.splint(0, x[n], tck)
    out += constant
    return out

In [ ]:
yint = integ(xnew, tck)
plt.figure()
plt.plot(xnew, yint, xnew, -np.cos(xnew), '--')
plt.legend(['Cubic Spline', 'True'])
plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Integral estimation from spline')
plt.show()

Y las raices de la spline

In [ ]:
x = np.linspace(-np.pi / 4, 2. * np.pi + np.pi / 4, 21)
y = np.sin(x)
tck = interpolate.splrep(x, y, s=0)
interpolate.sproot(tck)

Spline paramétrica

In [ ]:
interpolate.splprep?

In [ ]:
t = np.arange(0, 1.1, .1)
x = np.sin(2 * np.pi * t)
y = np.cos(2 * np.pi * t)
tck, u = interpolate.splprep([x, y], s=0)
unew = np.arange(0, 1.01, 0.01)
out = interpolate.splev(unew, tck)
plt.figure()
plt.plot(x, y, 'x', out[0], out[1], np.sin(
    2 * np.pi * unew), np.cos(2 * np.pi * unew))#, x, y, 'b')
plt.legend(['Linear', 'Cubic Spline', 'True'])
plt.axis([-1.05, 1.05, -1.05, 1.05])
plt.title('Spline of parametrically-defined curve')
plt.show()

## Representación de spline en dos dimensiones

Para un ajuste suave de una spline a una superficie en dos dimensiones, se usa la función 'bisplrep'.
Para evaluar la spline dos-dimensional y sus derivadas parciales, se usa la función 'bisplev'.

Se define una función sobre una grilla poco densa y la graficamos

In [ ]:
x, y = np.mgrid[-1:1:20j, -1:1:20j]
z = (x + y) * np.exp(-6.0 * (x * x + y * y))
plt.figure()
plt.pcolor(z)
plt.colorbar()
plt.title("Sparsely sampled function.")
plt.show()

Y se interpola sobre una grilla de 70x70, y se grafica

In [ ]:
xnew, ynew = np.mgrid[-1:1:70j, -1:1:70j]
tck = interpolate.bisplrep(x, y, z, s=0)
znew = interpolate.bisplev(xnew[:, 0], ynew[0, :], tck)
plt.figure()
plt.pcolor(znew)
plt.colorbar()
plt.title("Interpolated function.")
plt.show()

### Usando funciones de base radial  para suavizado/interpolación

Las funciones de base radial pueden ser usadas para suavizado/interpolación de datos dispersados en N-dimensiones, pero debe ser usada con precaución para extrapolación fuera del área de datos observados.

En el siguiente ejemplo se muestra cómo interpolar datos dispersos en 2D.

In [ ]:
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from matplotlib import cm
# 2-d tests - setup scattered data
x = np.random.rand(100) * 4.0 - 2.0
y = np.random.rand(100) * 4.0 - 2.0
z = x * np.exp(-x**2 - y**2)
ti = np.linspace(-2.0, 2.0, 100)
XI, YI = np.meshgrid(ti, ti)

In [ ]:
# use RBF
rbf = Rbf(x, y, z, epsilon=2)
ZI = rbf(XI, YI)
# plot the result
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI, cmap=cm.jet, shading='auto')
plt.scatter(x, y, 100, z, cmap=cm.jet)
plt.title('RBF interpolation - multiquadrics')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.colorbar()
plt.show()

# Algebra Lineal

'Scipy.linalg' contiene todas las funciones de 'Numpy.linalg', pero es más avanzada.

Las clases que representan matrices y operaciones básicas matriciales son parte de 'numpy'. Por conveniencia se resumen las diferencias entre 'numpy.matrix' y 'numpy.ndarray'.

'numpy.matrix' es una clase de matrices que tiene una mejor interfaz de operatoria que 'numpy.ndarray' para matrices. Esta clase tiene una sintaxis similar a la de Matlab, la multiplicación matricial por default es '*', posee los operadores inversa ('.I') y la traspuesta es ('.T').


In [ ]:
# Ejemplo de numpy.matrix
import numpy as np
A = np.mat('[1 2; 3 4]')
A
A.I
b = np.mat('[5 6]')
b.T
A * b.T

A pesar de su conveniencia, el uso de la clase 'numpy.matrix' no se recomienda, desde que no tiene nada que no se pueda lograr con arreglos 2-D de 'numpy.ndarray', y puede llevar a confusiones de cuál es la clase que se está usando, por ejemplo, el código antes mencionado puede ser escrito:

In [ ]:
A = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
b.T
A * b  # NO la multiplicación matricial
A.dot(b.T)  # Sí multiplicación matricial
# A.dot(b) #no importa por multiplicación trasponer o no.
b2 = np.array([5, 6])
A.dot(b2)

## Rutinas básicas

### Encontrando la inversa

In [ ]:
import numpy as np
from scipy import linalg

A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
print(A)
print(linalg.inv(A))

print(A.dot(linalg.inv(A)))  # double check

## Resolviendo un sistema lineal

De la forma:

\begin{align}
x + 3y + 5z = 10\\ 
2x + 5y +  z = 8\\ 
2x + 3y + 8z = 3
\end{align}

In [ ]:
import numpy as np
from scipy import linalg
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
print(A)

b = np.array([[10], [8], [3]])
print(b)

linalg.inv(A).dot(b)  # slow

A.dot(linalg.inv(A).dot(b)) - b  # check

np.linalg.solve(A, b)  # fast

A.dot(np.linalg.solve(A, b)) - b  # check

### Encontrando el determinante

In [ ]:
import numpy as np
from scipy import linalg
A = np.array([[1, 2], [3, 4]])
linalg.det(A)

## Calculando normas

Es posible calcular distintas normas, se presentan algunas normas matriciales a continuación

In [ ]:
import numpy as np
from scipy import linalg
A = np.array([[1, 2], [3, 4]])
A
linalg.norm(A)

linalg.norm(A, 'fro')  # frobenius norm is the default

linalg.norm(A, 1)  # L1 norm (max column sum)

linalg.norm(A, -1)

linalg.norm(A, np.inf)  # L inf norm (max row sum)

## Problema de los mínimos cuadrados 

El problema lineal de los minimos cuadrados aparece en muchas ramas de las matemáticas. En este problema un conjunto de coeficientes lineales se desean encontrar para poder ajustar el modelo a los datos.
En particular, se supone que los datos $y_i$ se relacionan con los datos $x_i$ por un conjunto de coeficientes $c_j$ y una función modelo $f_j(x_i)$ de la forma:

\begin{align}
y_i = \sum_{j} c_j f_j(x_i) + \epsilon_i
\end{align}

donde $\epsilon_i$ representa incertezas en los datos. La estrategia de los mínimos cuadrados es escoger coeficientes $c_j$ que minimicen el error cuadrático, es decir

\begin{align}
J(c) = \sum_{i} \left| y_i - \sum_j c_j f_j (x_i) \right| ^2
\end{align}

Definiendo $\{ A\}_{ij} = f_j (x_i)$, notemos que el problema se puede escribir $y = Ac + \epsilon$, y el problema posee solución cuando $A^H A$ es invertible y entonces

\begin{align}
c = \left( A^H A \right)^{-1} A^H y = A^{\dagger} y
\end{align}


El comando 'linalg.lstsq' resolverá el problema de los mínimos cuadrados para $c$ dado $A$ e $y$. Para calcular $A^{\dagger}$ se puede usar las funciones 'linalg.pinv' o 'linalg.pinv2'.

El siguiente ejemplo muestra el uso de las funciones antes mencionadas para el modelo

\begin{align}
y_i = c_1 e^{-x_i} + c_2 x_i
\end{align}
con $x_i=0.1$, $i = 1,...,10$, $c_1 =5, c_2 = 4$. Se añade ruido a $y_i$ y los coeficientes $c_1, c_2$ son estimados usando minimos cuadrados lineal

In [ ]:
# Least Square Method

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

In [ ]:
c1, c2 = 5.0, 2.0
i = np.r_[1:11]
xi = 0.1 * i
yi = c1 * np.exp(-xi) + c2 * xi
zi = yi + 0.05 * np.max(yi) * np.random.rand(len(yi))
print(yi)
print(zi)

In [ ]:
A = np.c_[np.exp(-xi)[:, np.newaxis], xi[:, np.newaxis]]
print(A)
c, resid, rank, sigma = linalg.lstsq(A, zi)

print(c, resid, rank, sigma)
print(c)

In [ ]:
xi2 = np.r_[0.1:1.0:100j]
yi2 = c[0] * np.exp(-xi2) + c[1] * xi2
plt.plot(xi, zi, 'x', xi2, yi2)
plt.axis([0, 1.1, 3.0, 5.5])
plt.xlabel('$x_i$')
plt.title('Data fitting with linalg.lstsq')
plt.show()

# Estadística

En esta sección se discuten algunos aspectos de 'scipy.stats'. 

## Variables Aleatorias

Hay dos clases de distribución que tienen implementadas variables aleatorias continuas y discretas. 
Todas las funciones estadísticas están alocadas en el subpaquete $\text{scipy.stats}$ y una lista de tales funciones se puede obtener vía 'info(stats)'.

Se importa el paquete de la siguiente forma:

In [ ]:
from scipy import stats
from scipy.stats import norm

Los métodos más usados para variables aleatorias continuas son:

    rvs: Random Variates
    pdf: Probability Density Function
    cdf: Cumulative Distribution Function
    sf: Survival Function (1-CDF)
    ppf: Percent Point Function (Inverse of CDF)
    isf: Inverse Survival Function (Inverse of SF)
    stats: Return mean, variance, (Fisher’s) skew, or (Fisher’s) kurtosis
    moment: non-central moments of the distribution


In [ ]:
# Por ejemplo, una variable aleatoria normal evaluada en ciertos puntos:
norm.cdf(np.array([-1., 0, 1.65]))

In [ ]:
# metodos generalmente útiles:
norm.mean(), norm.std(), norm.var()

In [ ]:
# Para encontrar la media de una distribución se puede usar la 'percent
# point function 'ppf'', la cual es la inversa de 'cdf'
norm.ppf(0.5)

## Kernel Density Estimation

Una tarea común en estadística es estimar la función de densidad de probabilidad (PDF) de una variable aleatoria a partir de un conjunto de muestra de datos. Esta tarea se denomina estimación de densidad (density estimation). La forma mejor conocida para hacer esto es un histograma, el cual es una herramienta útil de visualización, pero no usa los datos disponibles muy eficientemente. La estimación de la densidad de nucleo (kernel density estimation) es una herramienta más eficiente para la misma tarea.

El 'gaussian_kde' estimador puede ser usado para estimar la PDF de una univariada, así como una multivariada. Funciona mejor si los datos son unimodales.

### Estimación univariada

Se empieza con una cantidad mínima de datos para ver cómo 'gaussian_kde' funciona, y las diferentes opciones para ancho de banda hacen. Los datos sampleados desde PDF se muestran como guiones azules al fondo de la figura

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

x1 = np.array([-7, -5, 1, 4, 5], dtype=np.float32)
kde1 = stats.gaussian_kde(x1)
kde2 = stats.gaussian_kde(x1, bw_method='silverman')

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x1, np.zeros(x1.shape), 'b+', ms=20)  # rug plot
x_eval = np.linspace(-10, 10, num=200)
ax.plot(x_eval, kde1(x_eval), 'k-', label="Scott's Rule")
ax.plot(x_eval, kde2(x_eval), 'r-', label="Silverman's Rule")
plt.show()

Se ve que hay una pequeña diferencia entre 'Scott's Rule' y 'Silverman's Rule', y el ancho de banda escogido con una cantidad limitada de datos es probablemente muy ancha. Se puede definir una propia función de anchi de banda para obtener un resultado menos suavizado

In [ ]:
def my_kde_bandwidth(obj, fac=1. / 5):
    """We use Scott's Rule, multiplied by a constant factor."""
    return np.power(obj.n, -1. / (obj.d + 4)) * fac

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x1, np.zeros(x1.shape), 'b+', ms=20)  # rug plot
kde3 = stats.gaussian_kde(x1, bw_method=my_kde_bandwidth)
ax.plot(x_eval, kde3(x_eval), 'g-', label="With smaller BW")
plt.show()

Se ve que si el ancho de banda es muy angosto, se obtiene que el estimado de la función de densidad de probabilidad (PDF) es simplemente la suma de Gaussianas alrededor de cada uno de los puntos.

Tomemos ahora un ejemplo más realista y veamos la diferencia entre las dos elecciones de ancho de banda. Estas reglas son conocidas que funcionan bien para (o cercano a) distribuciones normales, pero incluso para distribuciones unimodales que son un poco fuertemente no-normales funcionan razonablemente bien.

Como una distribución no-normal tomaremos una T-Student distribución con 5 grados de libertad.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


np.random.seed(12456)
x1 = np.random.normal(size=200)  # random data, normal distribution
xs = np.linspace(x1.min() - 1, x1.max() + 1, 200)

kde1 = stats.gaussian_kde(x1)
kde2 = stats.gaussian_kde(x1, bw_method='silverman')

fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(211)
ax1.plot(x1, np.zeros(x1.shape), 'b+', ms=12)  # rug plot
ax1.plot(xs, kde1(xs), 'k-', label="Scott's Rule")
ax1.plot(xs, kde2(xs), 'b-', label="Silverman's Rule")
ax1.plot(xs, stats.norm.pdf(xs), 'r--', label="True PDF")

ax1.set_xlabel('x')
ax1.set_ylabel('Density')
ax1.set_title("Normal (top) and Student's T$_{df=5}$ (bottom) distributions")
ax1.legend(loc=1)

x2 = stats.t.rvs(5, size=200)  # random data, T distribution
xs = np.linspace(x2.min() - 1, x2.max() + 1, 200)

kde3 = stats.gaussian_kde(x2)
kde4 = stats.gaussian_kde(x2, bw_method='silverman')

ax2 = fig.add_subplot(212)
ax2.plot(x2, np.zeros(x2.shape), 'b+', ms=12)  # rug plot
ax2.plot(xs, kde3(xs), 'k-', label="Scott's Rule")
ax2.plot(xs, kde4(xs), 'b-', label="Silverman's Rule")
ax2.plot(xs, stats.t.pdf(xs, 5), 'r--', label="True PDF")

ax2.set_xlabel('x')
ax2.set_ylabel('Density')

plt.show()

Tomemos ahora una distrubución bimodal con una característica gaussiana ancha y una delgada. Esperamos que esto sea más dificil de aproximar, debido a las diferentes anchos de banda requeridos para resolver precisamente cada característica.


In [ ]:
from functools import partial

In [ ]:
loc1, scale1, size1 = (-2, 1, 175)
loc2, scale2, size2 = (2, 0.2, 50)
x2 = np.concatenate([np.random.normal(loc=loc1, scale=scale1, size=size1),
                     np.random.normal(loc=loc2, scale=scale2, size=size2)])

In [ ]:
x_eval = np.linspace(x2.min() - 1, x2.max() + 1, 500)

In [ ]:
kde = stats.gaussian_kde(x2)
kde2 = stats.gaussian_kde(x2, bw_method='silverman')
kde3 = stats.gaussian_kde(x2, bw_method=partial(my_kde_bandwidth, fac=0.2))
kde4 = stats.gaussian_kde(x2, bw_method=partial(my_kde_bandwidth, fac=0.5))

pdf = stats.norm.pdf
bimodal_pdf = pdf(x_eval, loc=loc1, scale=scale1) * float(size1) / x2.size + \
    pdf(x_eval, loc=loc2, scale=scale2) * float(size2) / x2.size

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

ax.plot(x2, np.zeros(x2.shape), 'b+', ms=12)
ax.plot(x_eval, kde(x_eval), 'k-', label="Scott's Rule")
ax.plot(x_eval, kde2(x_eval), 'b-', label="Silverman's Rule")
ax.plot(x_eval, kde3(x_eval), 'g-', label="Scott * 0.2")
ax.plot(x_eval, kde4(x_eval), 'c-', label="Scott * 0.5")
ax.plot(x_eval, bimodal_pdf, 'r--', label="Actual PDF")
ax.set_xlim([x_eval.min(), x_eval.max()])
ax.legend(loc=2)
ax.set_xlabel('x')
ax.set_ylabel('Density')
plt.show()

Como era de esperar, el KDE no es tan cercano como la verdadera PDF como se podría esperar debido a las diferentes caracteristicas de tamaño de las dos componentes de distribución bimodal.

Reduciendo a la mitad el ancho de banda defecto ('Scott*0.5') se puede lograr algo mejor, mientras que usando un factor 5 veces más pequeño que el ancho de banda por defecto no suaviza demasiado. Lo que se necesita en este caso es un ancho de banda no uniforme (adaptativo)

Usando la función 'gaussian_kde' se puede operar estimación multivariada al igual que la univariada. Se muestra a continuación un caso bivariado. En primer lugar se generan datos aleatorios con un modelo en que dos variables están correlacionadas.

In [ ]:
def measure(n):
    """Measurement model, return two coupled measurements."""
    m1 = np.random.normal(size=n)
    m2 = np.random.normal(scale=0.5, size=n)
    return m1 + m2, m1 - m2


m1, m2 = measure(2000)
xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()

In [ ]:
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([m1, m2])
kernel = stats.gaussian_kde(values)
Z = np.reshape(kernel.evaluate(positions).T, X.shape)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
          extent=[xmin, xmax, ymin, ymax])
ax.plot(m1, m2, 'k.', markersize=2)

ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

plt.show()

In [ ]:
np.r_?

In [ ]:
xx = np.r_[-3: 3: 1000j] #np.linspace(-3, 3, 1000)
plt.plot(xx, norm.cdf(xx))
plt.show()
norm.cdf(1)

In [ ]:
np.histogram(xx,10)
plt.hist(norm.rvs(size=100000),100)
plt.show()

In [ ]:
norm.stats(loc = 3, scale = 4, moments = 'mv')

In [ ]:
plt.hist(norm.rvs(size=100000, loc = 3, scale = 1000),100)
plt.show()

In [ ]:
from scipy.stats import uniform
uniform.cdf([0, 1, 2, 3, 4, 4], loc = 0.1)#, scale = 400)

In [ ]:
from scipy.stats import gamma
gamma.numargs
gamma.shapes

gamma(1, scale=2.).stats(moments="mv")

gamma(a=1, scale=2.).stats(moments="mv")

rv = gamma(1, scale=2.)

rv.mean(), rv.std()

In [ ]:
rgh = stats.gausshyper.rvs(0.5, 2, 2, 2, size=100)
print(rgh)

In [ ]:
from scipy import stats
class deterministic_gen(stats.rv_continuous):
    def _cdf(self, x):
        return np.where(x < 0, 0., 1.)
    def _stats(self):
        return 8., 0., 0., 8.

In [ ]:
deterministic = deterministic_gen(name="deterministic")
deterministic.cdf(np.arange(-3, 3, 0.5))
deterministic.pdf(np.arange(-3, 3, 0.5))

In [ ]:
help(stats.ttest_ind)

In [ ]:
rvs1 = stats.norm.rvs(loc=5, scale=10, size=500)
rvs2 = stats.norm.rvs(loc=5, scale=10, size=500)
stats.ttest_ind(rvs1, rvs2)